# Page 1-2

In [5]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.88 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora 39) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.150 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.155 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_1_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.198 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Gentoo) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.240 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.210 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.125 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.160 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.150 Safari/537.36"
]




random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/s?searchTerm=lip+care&tref=typeahead%7Cterm%7Clip+care%7C%7C%7Chistory"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ...前面的 import 和设置不变...

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list = []

def scrape_products():
    global product_list, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Cleansers",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page():
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
        time.sleep(random.uniform(8, 12))
        return True
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()
        if not go_to_next_page():
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")

driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (X11; Linux x86_64; Gentoo) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.240 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
🔁 检测到页面内容重复，疑似回到了第一页，停止抓取
⛔ 检测到重复页，主动中止爬虫
✅ WebDriver 关闭
📦 共抓取商品（去重后）：29 条


In [6]:
df = pd.DataFrame(product_list)
df

product_name              brand  \
0                 e.l.f. Squeeze Me Lip Balm - 0.21oz             e.l.f.   
1           Aquaphor Immediate Relief Lip Repair Balm           Aquaphor   
2   Aquaphor Lip Repair Stick for Dry Chapped Lips...           Aquaphor   
3   Vaseline Lip Therapy Mini Lip Balm - Rosy - Tr...           Vaseline   
4   Vaseline Rosy Lip Therapy Stick - 2pk/0.16oz each           Vaseline   
5   Vaseline Lip Therapy Advanced Healing Moisturi...           Vaseline   
6   Aquaphor Lip Repair Stick For Immediate Relief...           Aquaphor   
7   L'Oreal Paris Plump Ambition Hyaluron Lip Oil ...      L'Oreal Paris   
8   Maybelline Lifter Gloss Plumping Lip Gloss wit...         Maybelline   
9   eos Cashmere Lip Butter - Vanilla Cashmere - 0...                eos   
10           e.l.f. Holy Hydration! Lip Mask - 0.15oz             e.l.f.   
11  The Ordinary Squalane + Amino Acid Lip Balm - ...       The Ordinary   
12          eos Extra Dry Lip Repair Tube - 0.7 fl oz                eos   
13                 Sun Bum Lip Balm - SPF 30 - 0.15oz            Sun Bum   
14           e.l.f. Glow Reviver Lip Oil - 0.25 fl oz             e.l.f.   
15  La Roche Posay Cicaplast Lip Mini Balm - 0.25 ...     La Roche Posay   
16    eos Nilla Milkshake Lip Balm Stick - 0.56oz/4pk                eos   
17                        Starface Star Balm - 0.15oz           Starface   
18  Sport Sunscreen Lip Balm - SPF 50 - 0.3oz - up...              up&up   
19  eos Brown Sugar Bubble Scrub/Marshmallow Fluff...                eos   
20   eos Tropical Chiller Lip Balm Stick - 0.56oz/4pk                eos   
21   eos Sunrise Smoothie Lip Balm Stick - 0.56oz/4pk                eos   
22         Burt's Bees Lip Balm - Beeswax - 4ct/0.6oz        Burt's Bees   
23  Maybelline Lifter Gloss Plumping Lip Gloss wit...         Maybelline   
24  Vitamasques Tinted Lip Butter Balm Plump and R...        Vitamasques   
25          Aquaphor Immediate Relief Lip Repair Balm           Aquaphor   
26  eos Pink Paradise Mini Lip Balm Tube Blister -...                eos   
27  Carmex Classic Lip Balm Medicated Stick - 4pk/...             Carmex   
28  Florence by mills Hit Snooze Lip Mask - 0.37oz...  Florence by mills   

      current_price unit_price                           star_rating  \
0             $4.00        N/A  4.6 out of 5 stars with 2837 ratings   
1    $5.99 - $10.49        N/A  4.7 out of 5 stars with 5742 ratings   
2             $6.19        N/A  4.6 out of 5 stars with 2572 ratings   
3             $3.79        N/A  4.6 out of 5 stars with 3007 ratings   
4             $2.99        N/A  4.2 out of 5 stars with 1322 ratings   
5             $4.59        N/A  4.4 out of 5 stars with 1529 ratings   
6             $5.99        N/A  4.7 out of 5 stars with 1342 ratings   
7   $12.99 - $13.49        N/A   4.7 out of 5 stars with 473 ratings   
8    $7.64 - $11.49        N/A  4.6 out of 5 stars with 5288 ratings   
9             $7.89        N/A   3.4 out of 5 stars with 244 ratings   
10            $7.00        N/A  4.6 out of 5 stars with 1073 ratings   
11            $9.50        N/A   3.8 out of 5 stars with 856 ratings   
12            $9.29        N/A  4.6 out of 5 stars with 1118 ratings   
13            $5.49        N/A   3.6 out of 5 stars with 807 ratings   
14            $8.00        N/A  4.7 out of 5 stars with 8219 ratings   
15           $12.19        N/A   4.6 out of 5 stars with 500 ratings   
16           $12.19        N/A    4.8 out of 5 stars with 16 ratings   
17            $6.99        N/A   4.4 out of 5 stars with 792 ratings   
18            $5.39        N/A     4 out of 5 stars with 409 ratings   
19           $12.19        N/A    4.7 out of 5 stars with 14 ratings   
20           $12.19        N/A       5 out of 5 stars with 9 ratings   
21           $12.19        N/A    4.7 out of 5 stars with 19 ratings   
22           $11.79        N/A  4.7 out of 5 stars with 1909 ratings   
23   $7.64 - $11.49        N/A  

In [8]:
df.to_csv("lipcare_data.csv",index=False)

# Page 3-5

In [9]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.88 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora 39) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.150 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.155 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_1_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.198 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Gentoo) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.240 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.210 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.125 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.160 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.150 Safari/537.36"
]




random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/s?searchTerm=lip+care&tref=typeahead%7Cterm%7Clip+care%7C%7C%7Chistory&Nao=48&moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list2 = []

def scrape_products():
    global product_list2, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list2.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Cleansers",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page():
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
        time.sleep(random.uniform(8, 12))
        return True
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()
        if not go_to_next_page():
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")

driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list2)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.150 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
✅ WebDriver 关闭
📦 共抓取商品（去重后）：74 条


In [12]:
df2 = pd.DataFrame(product_list2)
df2.sample(5)

product_name        brand  \
43  Burt's Bees Party Pack - Birthday Cake Variety...  Burt's Bees   
17  Tree Hut Sugarlips Watermelon Lip Butter - 0.52oz     Tree Hut   
28  eos 24H Moisture Super Balm - Birthday Cake - ...          eos   
4   ESW Beauty Vanilla Almond Butter Smoothie Lip ...   ESW Beauty   
54  Chapstick Lip Moisturizer and Skin Protectant ...    Chapstick   

   current_price unit_price                          star_rating  \
43        $11.79        N/A   4.6 out of 5 stars with 84 ratings   
17         $8.29        N/A   4.5 out of 5 stars with 72 ratings   
28         $5.99        N/A   4.4 out of 5 stars with 19 ratings   
4         $14.69        N/A  4.9 out of 5 stars with 252 ratings   
54         $4.39        N/A  3.7 out of 5 stars with 277 ratings   

   review_counts  is_on_sale is_free_shipping  is_3day_delivery  \
43    84 reviews           0                0                 1   
17    72 reviews           0                0                 1   
28    19 reviews           0                0                 1   
4    252 reviews           0                0                 1   
54   277 reviews           0                0                 1   

   product_category                                       product_link  
43        Cleansers  https://www.target.com/p/burt-39-s-bees-party-...  
17        Cleansers  https://www.target.com/p/tree-hut-sugarlips-wa...  
28        Cleansers  https://www.target.com/p/eos-24h-moisture-supe...  
4         Cleansers  https://www.target.com/p/esw-beauty-vanilla-al...  
54        Cleansers  https://www.target.com/p/chapstick-lip-moistur...

In [11]:
df2.to_csv("lipcare_data2.csv",index=False)

# Page 6-7

In [18]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.15 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.15 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; openSUSE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.15 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.92 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_2_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.92 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Mint) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.92 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.132 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_9) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.132 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Debian) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.132 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Chrome/122.0.6261.160 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.160 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.160 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.199 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.199 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.199 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.250 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.250 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.250 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.221 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.221 Safari/537.36"
]



random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/s?searchTerm=lip+care&tref=typeahead%7Cterm%7Clip+care%7C%7C%7Chistory&Nao=120&moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 5))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list3 = []

def scrape_products():
    global product_list3, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list3.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Lip Care",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page():
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
        time.sleep(random.uniform(8, 12))
        return True
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()
        if not go_to_next_page():
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")

driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list3)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_9) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.132 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
❌ 没有找到下一页按钮，结束爬取
✅ WebDriver 关闭
📦 共抓取商品（去重后）：87 条


In [24]:
pd.set_option('display.max_rows', None)
df3 = pd.DataFrame(product_list3)
df3 = df3.iloc[:58]
df3


product_name  \
0   Lanolips Ultra Healing 101 Ointment Multi-Use ...   
1   Aquaphor Lip Repair Stick for Dry Chapped Lips...   
2   Jack Black Exclusive Lip Duo - 0.5oz/2ct - Ult...   
3         Versed Soft Gloss Tinted Lip Serum - 0.06oz   
4   Burt's Bees Strawberry Lemonade Lip Balm - 0.15oz   
5                Provence Beauty Tinted Glace - 0.2oz   
6         Burt's Bees Tinted Lip Balm - Rose - 0.15oz   
7          Burt's Bees Berry Medley Rescue Lip Relief   
8        Well People Lush Lip Tinted Oil - 0.14 fl oz   
9            Nivea Moisturizing Lip Balm - 0.34oz/2pk   
10           e.l.f. Glow Reviver Lip Oil - 0.25 fl oz   
11    Well People Lip Nurture Hydrating Balm - 0.28oz   
12  eos 100% Natural Lip Balm Sticks - Raspberry K...   
13   e.l.f. Glow Reviver Lip Oil Glimmer - 0.25 fl oz   
14  Carmex Weather Guard Broad Spectrum SPF 30 Ori...   
15  Buxom Full-On Plumping Lip Polish - 0.14oz - U...   
16  Neutrogena Hydro Boost Hydrating Lip Shine - 0...   
17    Avatara Guava Glow Tinted Lip Jelly - 0.5 fl oz   
18                Blistex Medex Lip Balm Sticks - 3pk   
19  eos 24H Moisture Super Balm - Raspberry Kiwi S...   
20      e.l.f. Glow Reviver Melting Lip Balm - 0.52oz   
21  Burt's Bees Moisturizing Lip Balm - Watermelon...   
22       Burt's Bees Boosted Tinted Lip Balm - 0.16oz   
23  Neutrogena Lip Gloss Moisture Shine Lip Soothe...   
24  Carmex Advanced Repair Moisturizing Lip Balm -...   
25    Chapstick You Are My Hero Lip Balm - 3ct/0.45oz   
26                 Bluey Egg Molded Lip Balm - 0.24oz   
27  Aquaphor Lip Balm Repair Stick for Chapped Lip...   
28  Lip Smacker Hello Kitty Ice Cream Cone Lip Bal...   
29    Burt's Bees Tinted Lip Balm - Hibiscus - 0.15oz   
30  NYX Professional Makeup Fat Oil Slick Click Ti...   
31         Burt's Bees Berry Medley Rescue Lip Relief   
32  bareMinerals Lip Gloss-Balm - 0.12 fl oz - Ult...   
33  Lanolips 101 Ointment Strawberry Mini Lip Ritu...   
34          Naturium Phyto Glow Lip Balm - 0.34 fl oz   
35              Burt's Bees Superfruit Lip Balm - 4ct   
36  L'Oreal Paris Colour Riche Original Satin Lips...   
37  Maybelline Lifter Gloss Lifter Plump Lip Plump...   
38  ESW Beauty Vanilla Almond Butter Smoothie Lip ...   
39  Mario Badescu Skincare Lip Mash with Vanilla a...   
40  Clinique Moisture Surge Lip Treatment - 0.34 f...   
41  Lanolips The Original 101 Ointment Multi Purpo...   
42  Jack Black Exclusive Lip Duo - 0.5oz/2ct - Ult...   
43           MCoBeauty Peptide Lip Treatment - 0.52oz   
44               MCoBeauty Overnight Lip Mask - 0.7oz   
45  Charlotte Tilbury Airbrush Flawless Lip Blur, ...   
46  Versed Cashmere Color Smoothing Lip Blush - 0....   
47  Buxom Power-Full Lip Scrub - Sweet Guava - Ult...   
48  Eos Lip Balm Smooth Sphere Strawberry Sorbet 0...   
49  eos 24H Moisture Super Balm - Honey Apple - 0....   
50         eos 24hrs Moisture Super Balm - 0.35 fl oz   
51  eos 100% Natural Watermelon Frosé Lip Balm Sti...   
52  Carmex Lip Balms and Treatments Classic Lip Ba...   
53  eos 24hrs Moisture Super Balm - Raspberry Kiwi...   
54  Blistex Medex Lip Tube Serum - Cherry - 0.35oz...   
55     COVERGIRL Outlast Vegan Lip Stain - 0.06 fl oz   
56  6ct Lip Balm Set Clear - Spritz™: Girls Lip Ba...   
57         MCoBeauty Hyaluronic Lip Balm - 0.14 fl oz   

                      brand    current_price unit_price  \
0                  Lanolips           $16.49        N/A   
1                  Aquaphor            $6.19        N/A   
2                Jack Black           $16.00        N/A   
3                    Versed           $11.99        N/A   
4               Burt's Bees            $3.99        N/A   
5           Provence Beauty            $8.79        N/A   
6               Burt's Bees            $5.99        N/A   
7               Burt's Bees            $5.99        N/A   
8               Well People           $12.00        N/A   
9                     NIVEA            $5.49        N/A   
10                   e.l.f. 

# 汇总前7页

In [25]:
df3.to_csv("lipcare_data3.csv",index=False)

In [27]:
df_combined_lipcare_1_7 = pd.concat([df, df2, df3], axis=0, ignore_index=True)
df_combined_lipcare_1_7

product_name  \
0                  e.l.f. Squeeze Me Lip Balm - 0.21oz   
1            Aquaphor Immediate Relief Lip Repair Balm   
2    Aquaphor Lip Repair Stick for Dry Chapped Lips...   
3    Vaseline Lip Therapy Mini Lip Balm - Rosy - Tr...   
4    Vaseline Rosy Lip Therapy Stick - 2pk/0.16oz each   
5    Vaseline Lip Therapy Advanced Healing Moisturi...   
6    Aquaphor Lip Repair Stick For Immediate Relief...   
7    L'Oreal Paris Plump Ambition Hyaluron Lip Oil ...   
8    Maybelline Lifter Gloss Plumping Lip Gloss wit...   
9    eos Cashmere Lip Butter - Vanilla Cashmere - 0...   
10            e.l.f. Holy Hydration! Lip Mask - 0.15oz   
11   The Ordinary Squalane + Amino Acid Lip Balm - ...   
12           eos Extra Dry Lip Repair Tube - 0.7 fl oz   
13                  Sun Bum Lip Balm - SPF 30 - 0.15oz   
14            e.l.f. Glow Reviver Lip Oil - 0.25 fl oz   
15   La Roche Posay Cicaplast Lip Mini Balm - 0.25 ...   
16     eos Nilla Milkshake Lip Balm Stick - 0.56oz/4pk   
17                         Starface Star Balm - 0.15oz   
18   Sport Sunscreen Lip Balm - SPF 50 - 0.3oz - up...   
19   eos Brown Sugar Bubble Scrub/Marshmallow Fluff...   
20    eos Tropical Chiller Lip Balm Stick - 0.56oz/4pk   
21    eos Sunrise Smoothie Lip Balm Stick - 0.56oz/4pk   
22          Burt's Bees Lip Balm - Beeswax - 4ct/0.6oz   
23   Maybelline Lifter Gloss Plumping Lip Gloss wit...   
24   Vitamasques Tinted Lip Butter Balm Plump and R...   
25           Aquaphor Immediate Relief Lip Repair Balm   
26   eos Pink Paradise Mini Lip Balm Tube Blister -...   
27   Carmex Classic Lip Balm Medicated Stick - 4pk/...   
28   Florence by mills Hit Snooze Lip Mask - 0.37oz...   
29           Milani Fruit Fetish Lip Oil - 0.135 fl oz   
30          Burt's Bees Vanilla Bean Lip Balm - 0.15oz   
31   bareMinerals Lip Gloss-Balm - 0.12 fl oz - Ult...   
32   Vitamasques Tinted Lip Butter Balm Plump and R...   
33   ESW Beauty Vanilla Almond Butter Smoothie Lip ...   
34          Sun Bum Lip Balm Tube - SPF 30 - 0.5 fl oz   
35               Burt's Bees Beeswax Lip Balm - 0.15oz   
36         Burt's Bees Lip Balm - Tropical Fruit - 4ct   
37     Avatara Guava Glow Tinted Lip Jelly - 0.5 fl oz   
38   Charlotte Tilbury Airbrush Flawless Lip Blur, ...   
39   Buxom Power-Full Lip Scrub - Sweet Guava - Ult...   
40   Aquaphor Lip Repair Stick For Immediate Relief...   
41         Bluey 7pk Lip Balm Set in Acetate Box - 7pk   
42   Chapstick Vanilla Favorites Lip Balm - Vanilla...   
43           Burt's Bees Beeswax Lip Balm - 2ct/0.15oz   
44       e.l.f. Glow Reviver Melting Lip Balm - 0.52oz   
45     Aquaphor Lip Protectant + SPF Lip Balm - 0.17oz   
46   Tree Hut Sugarlips Watermelon Lip Butter - 0.52oz   
47   Revlon Super Lustrous Glimmer Lip Oil - Moistu...   
48         eos Lip Balm Stick - Birthday Cake - 0.28oz   
49   Chapstick Classic Lip Balm - Original - 3ct/0....   
50        Burt's Bees Tinted Lip Balm - Daisy - 0.15oz   
51   Chapstick Lip Balm - Mocktail Collection - 0.1...   
52       NIVEA Hyaluronic Dewy Lip Care Stick - 0.18oz   
53   Vaseline Cocoa Butter Moisturizing Lip Therapy...   
54          Burt's Bees Vanilla Bean Lip Balm - 0.15oz   
55   Vitamasques Lip Mask and Repair - Plump - 0.13...   
56   Ulta Beauty Collection Juice Infused Lip Oil -...   
57   eos 24H Moisture Super Balm - Birthday Cake - ...   
58            MCoBeauty Peptide Lip Treatment - 0.52oz   
59   Aquaphor Lip Repair Stick For Immediate Relief...   
60               Burt's Bees Beeswax Lip Balm - 0.15oz   
61                Vaseline Lip Therapy Original 0.25oz   
62            Bluey Bingo Egg Molded Lip Balm - 0.24oz   
63   NYX Professional Makeup Fat Oil Slick Click Ti...   
64         Burt's Bees Lip Balm - Beeswax - 2pk/0.30oz   
65   NYX Professional Makeup Fat Oil Lip Drip Lip G...   
66       ESW Beauty Guava Mango Smoothie Lip Treatment   
67    Burt's Bees Lip Balm - Vanilla Bean - 2pk/0.30oz   
68   Tree Hut Sugarlips Strawberry Lip B

In [28]:
df_combined_lipcare_1_7.to_csv("lipcare_1_7.csv",index=False)

# Page 8-10

In [29]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.15 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.15 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; openSUSE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.15 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.92 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_2_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.92 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Mint) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.92 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.132 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_9) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.132 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Debian) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.132 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Chrome/122.0.6261.160 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.160 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.160 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.199 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.199 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.199 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.250 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.250 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.250 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.221 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6045.221 Safari/537.36"
]



random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/s?searchTerm=lip+care&tref=typeahead%7Cterm%7Clip+care%7C%7C%7Chistory&Nao=168&moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 5))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list4 = []

def scrape_products():
    global product_list4, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list4.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Lip Care",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page():
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
        time.sleep(random.uniform(8, 12))
        return True
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()
        if not go_to_next_page():
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")

driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list4)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (X11; Linux x86_64; Mint) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.92 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
✅ WebDriver 关闭
📦 共抓取商品（去重后）：58 条


In [30]:
df4 = pd.DataFrame(product_list4)
df4

product_name  \
0     e.l.f. Squeeze Me More Lip Balm Duo - 2 /0.21oz   
1         Versed Soft Gloss Tinted Lip Serum - 0.06oz   
2       Burt's Bees Passionfruit Lip Oil - 0.27 fl oz   
3          MCoBeauty Hyaluronic Lip Balm - 0.14 fl oz   
4         Vitamasques Glow Lip Oil - Strawberry Glaze   
5   Chapstick Active 2-in-1 Unscented Lip Balms - ...   
6       Wet n Wild Chameleon Chrome Lip Foil - 0.08oz   
7   Ulta Beauty Collection Weightless Water Lip St...   
8   Ulta Beauty Collection Cloud Kiss Matte Lip Cr...   
9      Blistex Repair & Protect Lip Balm Sticks - 2pk   
10   Winky Lux Glimmer Balm Lip Stain - Rosé - 0.13oz   
11   DHC Deep Moisturizing Lip Cream - Clear - 0.05oz   
12  Burt's Bees Lip Balm Ultra Conditioning Bliste...   
13    Burt's Bees Lip Plumping Serum - Clear - 0.27oz   
14  Chapstick Moisturizer Lip Balm - Original/Blac...   
15  Burt's Bees Shea + Coconut Oil Paper Tube Lip ...   
16  Revlon Super Lustrous Glass Shine Tinted Lip B...   
17        Burt's Bees Lip Balm - Pomegranate - 0.15oz   
18           Pacifica Lavender Moon Lip Mask - 0.63oz   
19  bareMinerals Lip Gloss-Balm - 0.12 fl oz - Ult...   
20              MCoBeauty Coco Lip Treatment - 0.21oz   
21  ESW Beauty Vanilla Almond Butter Smoothie Lip ...   
22  TULA SKINCARE 24/7 Moisture Day & Night Peptid...   
23  ColourPop Snow White Just One Bite Lip Scrub -...   
24  Versed Cloud Polish Exfoliating Lip Scrub - 0....   
25    Fourth Ray Alice Pure Madness Lip Mask - 0.25oz   
26    NatureWell Hydrating Lip Mask - Vanilla - 0.7oz   
27  Dionis Goat Milk Lip Balm Peppermint Twist, .2...   
28   ColourPop Belle Enchanted Rose Lip Mask - 0.39oz   
29             e.l.f. Lip Plumping Gloss - 0.09 fl oz   
30  NYX Professional Makeup Fat Oil Slick Click Ti...   
31  Versed Cloud Polish Exfoliating Lip Scrub - 0....   
32    Fourth Ray Alice Pure Madness Lip Mask - 0.25oz   
33    NatureWell Hydrating Lip Mask - Vanilla - 0.7oz   
34  Versed Cashmere Color Smoothing Lip Blush - 0....   
35  ESW Beauty Vanilla Almond Butter Smoothie Lip ...   
36  NYX Professional Makeup Duck Plump High Pigmen...   
37  Neutrogena Lip Gloss Moisture Shine Lip Soothe...   
38  Dionis Goat Milk Lip Balm Peppermint Twist, .2...   
39   ColourPop Belle Enchanted Rose Lip Mask - 0.39oz   
40                   Tatcha The Kissu Lip Mask 0.3 oz   
41                    Weleda Everon Lip Balm - 0.17oz   
42  Chapstick Moisturizer Lip Balm - Original - SP...   
43  ESW Beauty Vanilla Almond Butter Smoothie Lip ...   
44  TULA SKINCARE Rose Glow + Get It Lip Treatment...   
45  Burt's Bees Rescue Squeeze + Honey Bundle Lip ...   
46  Image Skincare Ormedic Balancing Lip Enhanceme...   
47  Charlotte Tilbury Airbrush Flawless Lip Blur, ...   
48  Lanolips Ultra Healing Lemonaid Lip Balm Treat...   
49      NatureWell Hydrating Lip Mask - Berry - 0.7oz   
50  Buxom Power-Full Lip Scrub - Sweet Guava - Ult...   
51      ColourPop Fresh Kiss Glossy Lip Balm - 0.49oz   
52  Lip Smacker Best Flavor Forever Lip Balm Party...   
53  Burt's Bees Lip Balm Sweet Peach Blister Box -...   
54  Neosporin Lip Health Overnight Renewal Therapy...   
55   Provence Beauty Soft Bisous Lip Mask - 0.5 fl oz   
56         MCoBeauty Hyaluronic Lip Balm - 0.14 fl oz   
57                        Lanolips Minty 101 Ointment   

                        brand    current_price unit_price  \
0                      e.l.f.            $8.00        N/A   
1                      Versed           $11.99        N/A   
2                 Burt's Bees            $7.59        N/A   
3                   MCoBeauty            $8.99        N/A   
4                 Vitamasques            $6.44        N/A   
5                   Chapstick            $9.79        N/A   
6                  Wet n Wild            $7.39        N/A   
7      Ulta Beauty Collection           $11.00        N/A   
8      Ulta Beauty Collection           $12.00        N/A   
9                     Blistex            $7.99        N/A   
10    

In [31]:
df4.to_csv("lipcare_8_10.csv",index=False)